In [ ]:
pip install openpyxl

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("SI2M-Lab/DarijaBERT")
model = AutoModelForMaskedLM.from_pretrained("SI2M-Lab/DarijaBERT")
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/307 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/836M [00:00<?, ?B/s]

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(80000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [ ]:
import pandas as pd

ds = pd.read_excel(r'/content/sample_data/neg_comments.xlsx')
comments = ds['Comment'].tolist()

# ***3. Random Insertion***

In [ ]:
caracters = {"&", "/", ".", ",", ":", ";", "!", "?", "-", "_", "'", '"', "(", ")", "[", "]", "{", "}"}

def not_str_token(token):
    #sous_tokens
    if token.startswith("##"):
        return True
    if token.endswith("##"):
        return True
    if token.isdigit():
        return True
    if not re.fullmatch(r'[\u0600-\u06FF]+', token):
        return True
    # caractères
    if token in caracters:
        return True
    #emoticons
    if all(not c.isalnum() for c in token):
        return True
    return False

In [ ]:
from transformers import pipeline
import re
import random
import torch

pipe = pipeline("fill-mask", model="SI2M-Lab/DarijaBERT")

def get_predicted_commentsByInsertion(comment):

  comment = re.sub(r'[^\w\s]', '', comment)
  previous_word = " "
  next_word = " "
  comment_words = comment.split()
  #print(comment_words)

  # Sélection d'un indice aléatoire
  pos = random.randint(0, len(comment_words) - 1)
  if pos != 0:
    previous_word=comment_words[pos-1]

  elif pos != len(comment_words)-1 :
    next_word=comment_words[pos+1]

  #ajouter token maské
  comment_words.insert(pos,"[MASK]")

  input_comment = ' '.join(comment_words)
  #print(input_comment)

  resultats = pipe(input_comment,top_k=5)
  step1 = pipe(input_comment,top_k=15)[0]['sequence']
  comment_variances = []

  filtered = [result for result in resultats
              if not not_str_token(result['token_str'])
              and result['token_str'] != next_word
              and result['token_str'] != previous_word]

  for f in filtered:
     comment_variances.append(f['sequence'])
     #print(f"{f['sequence']} → score: {f['score']:.4f}")
  return comment_variances

In [ ]:
def get_comment_embedding(comment):

    inputs = tokenizer(comment, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)
    model.config.output_hidden_states = True
    embeddings = outputs.hidden_states[-1]  # [batch_size, seq_len, hidden_dim]

    # Ignorer CLS et SEP et prendre la moyenne des embeddings restants
    mask = inputs["attention_mask"].unsqueeze(-1)  # [batch_size, seq_len, 1]
    sum_embeddings = torch.sum(embeddings * mask, dim=1)
    mean_embedding = sum_embeddings / mask.sum(dim=1)  # Moyenne pondérée

    #Retourne l'embedding moyen d'une phrase après passage par le modèle darija bert
    return mean_embedding.squeeze(0)  # Retirer la dimension batch

In [ ]:
#maintenant qu'on 5 variantes du commentaire, on va calculer cosin_dist entre le commentaire originale et les phrases augmentées
#on va par la suite conserver les phrases ayant une cos_dist sup à 80%
from torch.nn.functional import cosine_similarity


def get_Best_Variances(comment,comment_variances):

    original_embedding = get_comment_embedding(comment)
    best_score=0
    best_comment=comment
    for cmt in comment_variances:

        cmt_vr_embedding = get_comment_embedding(cmt)
        score = cosine_similarity(original_embedding, cmt_vr_embedding,dim=0).item()

        #print("variant", cmt, "score = ", score)
        if score > best_score:
            best_score = score
            #print(score)
            best_comment = cmt
    if best_score >= 0.90:
      return best_comment
    else:
      return ""


In [ ]:
def random_insertion(comment):
  comment_variances = get_predicted_commentsByInsertion(comment)
  augmented_comment = get_Best_Variances(comment,comment_variances)
  return augmented_comment

In [ ]:
augmentedComments_by_RI =[]
for comment in comments:
    augmentedcomment = random_insertion(comment)
    if augmentedcomment != "":
      augmentedComments_by_RI.append(augmentedcomment)

In [ ]:
RI_data = { "Augmented Comment": augmentedComments_by_RI}
df_RI = pd.DataFrame(RI_data)

In [ ]:
df_RI.to_excel('augmentednegcomments_byRI.xlsx')